In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.get('https://www.kotak.com/en/help-center.html')
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<link href="/etc/designs/kotak/clientlib-site/images/favicon.ico" rel="icon" sizes="16x16" type="image/vnd.microsoft.icon"/>
<meta content="text/html; charset=utf-8; IE=edge" http-equiv="content-type"/>
<link href="https://www.kotak.com/en/help-center.html"/>
<link href="https://www.kotak.com/en/help-center.html" rel="canonical"/>
<link as="font" crossorigin="" href="https://www.kotak.com/etc/designs/kotak/clientlib-site/fonts/svg-icon/icomoon.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://www.kotak.com/etc/designs/kotak/clientlib-site/fonts/Roboto-fonts/Roboto-Bold.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://www.kotak.com/etc/designs/kotak/clientlib-site/fonts/Roboto-fonts/Roboto-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://www.kotak.com/etc/designs/kotak/clientlib-site/fonts/Roboto-fonts/Roboto-Light.woff2" rel="pr

In [5]:
category = soup.find_all('div', attrs={'class': 'FAQ-list-content FAQ-list-bottom'})

In [6]:
category [0]

<div class="FAQ-list-content FAQ-list-bottom">
<div class="FAQ-list">
<ul class="self-span">
<div class="FAQ-list-item">
<a href="/en/help-center/bank-account.html"><b> <span style="font-size: 16.0px;">Account</span></b><br/>
Debit Card, ATM, Chequebook request, Balance, KYC or Re-KYC etc<i class="FAQ-list-right"></i></a>
</div>
</ul>
</div>
</div>

In [8]:
category [0].text.strip()

'Account\nDebit Card, ATM, Chequebook request, Balance, KYC or Re-KYC etc'

In [11]:
category [0].find("a").get("href")

'/en/help-center/bank-account.html'

In [12]:
base_url = 'https://www.kotak.com'

In [14]:
response2 = requests.get(f"{base_url}{category[0].find('a').get('href')}")

In [15]:
soup2 = BeautifulSoup(response2.text, 'html.parser')

In [16]:
soup2


<!DOCTYPE html>

<html lang="en">
<head>
<link href="/etc/designs/kotak/clientlib-site/images/favicon.ico" rel="icon" sizes="16x16" type="image/vnd.microsoft.icon"/>
<meta content="text/html; charset=utf-8; IE=edge" http-equiv="content-type"/>
<link href="https://www.kotak.com/en/help-center/bank-account.html"/>
<link href="https://www.kotak.com/en/help-center/bank-account.html" rel="canonical"/>
<link as="font" crossorigin="" href="https://www.kotak.com/etc/designs/kotak/clientlib-site/fonts/svg-icon/icomoon.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://www.kotak.com/etc/designs/kotak/clientlib-site/fonts/Roboto-fonts/Roboto-Bold.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://www.kotak.com/etc/designs/kotak/clientlib-site/fonts/Roboto-fonts/Roboto-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://www.kotak.com/etc/designs/kotak/clientlib-site/fonts/Roboto-fonts/R

In [17]:
sub_category = soup2.find_all('div', attrs={'class': 'FAQ-list-content FAQ-list-bottom'})

In [18]:
sub_category [0]

<div class="FAQ-list-content FAQ-list-bottom">
<div class="FAQ-list">
<ul class="self-span">
<div class="FAQ-list-item">
<a href="/en/help-center/bank-account/kyc-and-re-kyc.html">KYC and Re-KYC for Bank Account<i class="FAQ-list-right"></i></a>
</div>
</ul>
</div>
</div>

In [27]:
extracted_data = {
    "data": []
}

In [24]:
def response_status(response: requests.Response):
    if response.status_code == 200:
        return True
    else:
        raise

In [23]:
def check_faq_landing(soup: BeautifulSoup):
    if soup.find('div', attrs={'class': 'FAQ-landing'}) is not None:
        return True
    else:
        return False

In [28]:
import time

In [33]:
def scrap_faq(url):
    time.sleep(0.25)
    response = requests.get(url)
    if response_status(response):
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        if check_faq_landing(soup):
            
            query = soup.find('ul', attrs={'class': 'breadcrumb self-span'}).text.strip()
            
            answer = (soup.find('div', attrs={'class': 'FAQ-landing'})
                      .find('div', attrs={'class': 'FAQ-landing-content'})
                      .find('div', attrs={'class': 'selfhelpanswer self-span answerself-help'})
                      .text.strip())
            extracted_data["data"].append(f"{query}\n{answer}")
        
        else:
            base_url = 'https://www.kotak.com'
            for url in soup.find_all('div', attrs={'class': 'FAQ-list-content FAQ-list-bottom'}):
                url_ = url.find('a').get('href')
                scrap_faq(f"{base_url}{url_}")
    

In [34]:
main_url = 'https://www.kotak.com/en/help-center.html'

In [38]:
scrap_faq(main_url)

KeyboardInterrupt: 

In [39]:
extracted_data["data"]

['Why Re-KYC is required?\nKYC (Know Your Customer) details are updated while opening the Account.\nBased on RBI guidelines, Bank may ask for Re-KYC at specific intervals to keep the records of the Bank updated. If any of your personal or contact information has changed, they will updated in Bank Records through Re-KYC process.',
 "What are the documents required for Re-KYC?\nAs per the RBI norms Re-KYC needs to done for all customers at specific intervals. Bank may request the customer to update their latest contact details and correspondence address with supporting documents.\nFollowing documents can be submitted for Re-KYC (any one document as Proof of Identify and one for Proof of Address):\n\nPassport\nVoter's Identity Card\nDriving License\nAadhaar Letter/Card\n\nAll documents should be self-attested.",
 'What is the reason for my KYC not being updated?\nIf you have completed your full KYC process through the biometric process, it may take up to 48 working hours to get updated. I